#TP4 NLP: 2 finetunes em um Modelo



----
Aluno PPGI: Igor de Souza Lima

O notebook do Unsloth foi usado como base para esse exercício.


### Installation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.5: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.6.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass


In [ ]:
import json
json_importado = json.load(open('dados/train_spider.json')) #verificar se o caminho condiz para onde o senhor baixou

dados_para_finetuning = []
cont = 0

for item in json_importado:
    pergunta = item["question"]
    resposta = item["query"]
    cont+=1

    data = {
        "pergunta": pergunta,
        "resposta": resposta
    }

    dados_para_finetuning.append(data)

with open("question_answer_finetune.json", "w", encoding='utf-8') as arquivo_json:
    json.dump(dados_para_finetuning, arquivo_json, indent=4, ensure_ascii=False)

In [ ]:
import json

def json_para_sharegpt_jsonl(input_json_path, output_jsonl_path):
    """
    Lê o arquivo JSON de entrada (que é uma lista de objetos) e o converte
    para o formato JSONL (um JSON por linha) no padrão ShareGPT.
    """
    try:
        # Carrega o arquivo JSON inteiro de uma vez
        with open(input_json_path, "r", encoding="utf-8") as f:
            dados_json = json.load(f)

        # Abre o arquivo de saída para escrever linha por linha
        with open(output_jsonl_path, "w", encoding="utf-8") as out_f:
            # Itera sobre cada item (dicionário) da lista carregada
            for item in dados_json:
                pergunta = item.get("pergunta")
                resposta = item.get("resposta")

                # Garante que os campos "pergunta" e "resposta" existem
                if pergunta is None or resposta is None:
                    continue

                # Cria a estrutura de conversação do ShareGPT
                conversa = [
                    {"from": "user",      "value": pergunta},
                    {"from": "assistant", "value": resposta}
                ]

                # Cria a linha JSON final e a escreve no arquivo .jsonl
                linha_json = {"conversations": conversa}
                out_f.write(json.dumps(linha_json, ensure_ascii=False) + "\n")

    except FileNotFoundError:
        print(f"Erro: O arquivo de entrada '{input_json_path}' não foi encontrado.")
    except json.JSONDecodeError:
        print(f"Erro: O arquivo '{input_json_path}' não é um JSON válido.")

if __name__ == "__main__":
    arquivo_txt  = "/content/question_answer_finetune.json"
    arquivo_jsonl = "qa_sharegpt.jsonl"
    json_para_sharegpt_jsonl(arquivo_txt, arquivo_jsonl)
    print(f"Arquivo gerado: {arquivo_jsonl}")


Arquivo gerado: qa_sharegpt.jsonl


In [ ]:
from unsloth.chat_templates import standardize_sharegpt
from datasets import load_dataset

dataset = load_dataset("json", data_files="qa_sharegpt.jsonl", split="train")
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [ ]:
dataset[5]["conversations"]

[{'content': 'What are the names of the heads who are born outside the California state?',
  'role': 'user'},
 {'content': "SELECT name FROM head WHERE born_state != 'California'",
  'role': 'assistant'}]

In [ ]:
dataset[5]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat are the names of the heads who are born outside the California state?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSELECT name FROM head WHERE born_state != 'California'<|eot_id|>"

## Treinamento do Modelo: versão 1

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/7000 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/7000 [00:00<?, ? examples/s]

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.train_dataset[500]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHow many students does each advisor have?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSELECT advisor,  count(*) FROM Student GROUP BY advisor<|eot_id|>'

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[500]["labels"]])

'                                            SELECT advisor,  count(*) FROM Student GROUP BY advisor<|eot_id|>'

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
3.441 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,000 | Num Epochs = 1 | Total steps = 875
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Step,Training Loss
1,0.705700
2,0.643400
3,0.429400
4,0.485400
5,0.487000
6,0.503000
7,0.375900
8,0.511000
9,0.404800
10,0.456000


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1705.8537 seconds used for training.
28.43 minutes used for training.
Peak reserved memory = 3.441 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 23.343 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": '''Your task is to translate the natural language question into a SQL query, based on the following high-quality examples. Pay attention to how the questions relate to the tables, columns, joins, and aggregations in the resulting SQL query.
                                    ### Example 1
                                    Question: Return the average, maximum, and minimum budgets in millions for movies made before the year 2000.
                                    SQL: SELECT avg(budget_million) , max(budget_million) , min(budget_million) FROM movie WHERE YEAR < 2000
                                    ### Example 2
                                    Question: List all company names with a book published by Alyson.
                                    SQL: SELECT T1.company_name FROM culture_company AS T1 JOIN book_club AS T2 ON T1.book_club_id = T2.book_club_id WHERE T2.publisher = 'Alyson'
                                    ### Example 3
                                    Question: What are the titles, years, and directors of all films, ordered by budget in millions?
                                    SQL: SELECT title , YEAR , director FROM movie ORDER BY budget_million
                                    ### Final Task
                                    Question: How many heads of the departments are older than 56 ?
                                    SQL:'''},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 0.1, min_p = 0.1)

SELECT count(*) FROM department AS T1 JOIN staff AS T2 ON T1.department_id = T2.department_id WHERE T2.age  >  56<|eot_id|>


In [ ]:
model.save_pretrained("lora_model_version1")  # Local saving
tokenizer.save_pretrained("lora_model_version1")

('lora_model_version1/tokenizer_config.json',
 'lora_model_version1/special_tokens_map.json',
 'lora_model_version1/tokenizer.json')

## Treinamento do Modelo: versão 2

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 3, #mudei aqui
        # max_steps = 60,
        learning_rate = 2e-8, #mudei aqui
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 8000, #mudei aqui
        output_dir = "outputs",
        report_to = "none",
    ),
)

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.train_dataset[500]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHow many students does each advisor have?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSELECT advisor,  count(*) FROM Student GROUP BY advisor<|eot_id|>'

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[500]["labels"]])

'                                            SELECT advisor,  count(*) FROM Student GROUP BY advisor<|eot_id|>'

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
3.441 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,000 | Num Epochs = 3 | Total steps = 1,311
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Step,Training Loss
1,0.142900
2,0.131300
3,0.122000
4,0.179900
5,0.096200
6,0.099100
7,0.096500
8,0.165700
9,0.118700
10,0.089400


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3798.2832 seconds used for training.
63.3 minutes used for training.
Peak reserved memory = 3.441 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 23.343 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": '''Your task is to translate the natural language question into a SQL query, based on the following high-quality examples. Pay attention to how the questions relate to the tables, columns, joins, and aggregations in the resulting SQL query.
                                    ### Example 1
                                    Question: Return the average, maximum, and minimum budgets in millions for movies made before the year 2000.
                                    SQL: SELECT avg(budget_million) , max(budget_million) , min(budget_million) FROM movie WHERE YEAR < 2000
                                    ### Example 2
                                    Question: List all company names with a book published by Alyson.
                                    SQL: SELECT T1.company_name FROM culture_company AS T1 JOIN book_club AS T2 ON T1.book_club_id = T2.book_club_id WHERE T2.publisher = 'Alyson'
                                    ### Example 3
                                    Question: What are the titles, years, and directors of all films, ordered by budget in millions?
                                    SQL: SELECT title , YEAR , director FROM movie ORDER BY budget_million
                                    ### Final Task
                                    Question: How many heads of the departments are older than 56 ?
                                    SQL:'''},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 0.3, min_p = 0.9)

SELECT count(*) FROM department AS T1 JOIN staff AS T2 ON T1.department_id = T2.department_id WHERE T2.age > 56<|eot_id|>


<a name="Save"></a>
### Saving, loading finetuned models

In [ ]:
model.save_pretrained("lora_model_version2")  # Local saving
tokenizer.save_pretrained("lora_model_version2")

('lora_model_version2/tokenizer_config.json',
 'lora_model_version2/special_tokens_map.json',
 'lora_model_version2/tokenizer.json')